# Iterované vězňovo dilema

In [11]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from utils import play, write_best_genom, load_best_genom
import strategies
import inspect

## Strategie odpovědí
obecně jde o funkci tvaru *zrada(moje_historie_tahu, druheho_historie_tahu)* pro seznam minulých rozhodnutí vrátí, zda protihráče v následujícím tahu zradíme (1) nebo ne (0)

- první parametr je seznam historie našich tahů
- druhý parametr je seznam historie tahů protihráče


In [12]:
# vždy kooperuje
def always_cooperate(my_history, other_history):
    return 0

# náhodná odpověď
def random_answer(my_history, other_history):
    p = random.random()
    if p < 0.5:
        return 1

    return 0

# oko za oko -> pokud druhý vezeň zradí, tak následující tah ho také zradí
def tit_for_tat(my_histroy, other_history):

    if len(my_histroy) == 0:
        return 0

    elif other_history[-1] == 1:
        return 1

    else:
        return 0

# vždy zradí
def always_defect(my_history, other_history):
    return 1

# zradí po dvou zradách
def tit_for_two_tats(my_history, other_history):
    if len(my_history) < 2:
        return 0
    elif other_history[-1] == 1 and other_history[-2] == 1:
        return 1
    else:
        return 0

# spolupracuje, dokud soupeř nezradí – pak už vždy zrazuje
def grim_trigger(my_history, other_history):
    return 0 if 1 not in other_history else 1

# Náhodná strategie s předsudkem (80 % spolupráce)
def biased_random(my_histroy, other_history):
    return 0 if random.random() < 0.8 else 1

# Pavlov (Win-Stay, Lose-Shift)
def pavlov(my_history, other_history):
    if len(my_history) == 0:
        return 0
    if my_history[-1] == other_history[-1]:  # pokud výsledek byl shodný, pokračuj
        return my_history[-1]
    else:
        return 1 - my_history[-1]  # změň tah

# Strategie testuje reakce soupeře
def probing(my_history, other_history):
    if len(my_history) < 3:
        return [0, 1, 1][len(my_history)]  # testuje reakci
    elif other_history[1] == 0:
        return 1  # pokud soupeř byl příliš důvěřivý, zrazuje
    else:
        return other_history[-1]  # jinak napodobuje

# Zrazuje pouze 2 kola poté co byl zrazen
def soft_grudger(my_history, other_history):
    punishment = 2
    if 1 in other_history:
        last_betray = len(other_history) - 1 - other_history[::-1].index(1)
        if len(other_history) - last_betray <= punishment:
            return 1
    return 0

# Zrazuje střídavě
def alternator(my_history, other_history):
    return len(my_history) % 2

# Oko za oko s procentem odpuštění
def generous_tit_for_tat(my_history, other_history):
    if len(other_history) == 0:
        return 0
    if other_history[-1] == 1:
        return 0 if random.random() < 0.1 else 1
    return 0


## Pomocné funkce

Pomocné funkce, které zjednodušují čitelnost a znovupoužitelnost kódu

In [13]:
import json
import time
import os

def rozdej_skore(tah1, tah2):
    # 1 = zradi, 0 = nezradi

    skores = (0, 0)

    if (tah1 == 1) and (tah2 == 1):
        skores = (2, 2)

    if (tah1 == 1) and (tah2 == 0):
        skores = (0, 3)

    if (tah1 == 0) and (tah2 == 1):
        skores = (3, 0)

    if (tah1 == 0) and (tah2 == 0):
        skores = (1, 1)

    return skores


def play(f1, f2, stepsnum):

    skore1 = 0
    skore2 = 0

    historie1 = []
    historie2 = []

    for i in range(stepsnum):
        tah1 = f1(historie1, historie2)
        tah2 = f2(historie2, historie1)

        s1, s2 = rozdej_skore(tah1, tah2)
        skore1 += s1
        skore2 += s2

        historie1.append(tah1)
        historie2.append(tah2)

    return skore1, skore2

def write_best_genom(genom, participants, score):

    data = {
        "created_at": f"{int(time.time())}",
        "genom": genom,
        "trained_on": [i.__name__ for i in participants],
        "score": score
    }

    if os.path.getsize("best_genom.json") > 0 and os.path.exists("best_genom.json"):
        with open("best_genom.json", "r") as f:
            loaded_data = json.load(f)

        with open("genom_history.json", "r") as f:
            history_data = json.load(f)

        if score < loaded_data["score"]:
            with open("genom_history.json", "w") as f:
                history_data["history"].append(loaded_data)
                f.write(json.dumps(history_data, indent=4, sort_keys=True, default=str))

            with open("best_genom.json", "w") as f:
                f.write(json.dumps(data, indent=4, sort_keys=True, default=str))

        if score > loaded_data["score"]:
            with open("genom_history.json", "w") as f:
                history_data["history"].append(data)
                f.write(json.dumps(history_data, indent=4, sort_keys=True, default=str))

    else:
        with open("best_genom.json", "w") as f:
            f.write(json.dumps(data, indent=4, sort_keys=True, default=str))

def load_best_genom():
    with open("best_genom.json", "r") as f:
        data = json.load(f)
        return data

## Evoluce strategie

Evoluce hledá nejlepší genom, který funkce `zrada` využívá. Evoluce spustí 200 individuálů po 100 generácí. Pracuje s crossoverem (50%) a mutací (20%).

Pokud najde lepší genom než ten, který je zapsaný v best_genom.json tak přepíše genom a starý genom zapíše do genom_history.json. Pokud není lepší než ten, který máme tak se zapíše do historie.


In [14]:

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 16)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# načtení všech strategii
test_ucastnici = [
    f for name, f in vars(strategies).items()
    if inspect.isfunction(f)
       and f.__module__ == 'strategies'
]

# Strategie pro evoluci k získání "nejlepšího" genomu
def strategy_from_genom(genom):
    def strategy(my_history, opponent_history):
        round_num = len(my_history)

        # 1. Úvodní fáze: poznávání soupeře
        if round_num < 3:
            return 0  # spolupracuj pro navázání důvěry
        elif round_num < 10:
            if opponent_history[-1] == 1:
                return 1  # test: pokud mě zradí brzo, odpověz zradou
            return 0  # jinak dál spolupracuj

        # 2. Detekce defektora
        recent_defections = opponent_history[-5:].count(1)
        total_defections = opponent_history.count(1)
        if total_defections / round_num > 0.3 and recent_defections >= 3:
            return 1  # zraď, pokud je soupeř defektor

        # 3. Odpouštění
        if opponent_history[-2:] == [0, 0]:
            return 0  # odpustíme po 2 spolupracích

        # 4. Genomová tabulka (paměť 2 tahy zpět)
        idx = 8 * my_history[-2] + 4 * my_history[-1] + 2 * opponent_history[-2] + opponent_history[-1]
        return genom[idx]
    return strategy

# fitness function, která vrátí skóre, které potomek získal
def fitness_function(individual):
    moje_strategie = strategy_from_genom(individual)
    total_score = 0
    for protivnik in test_ucastnici:
        s, _ = play(moje_strategie, protivnik, 200)
        total_score += s
    return total_score / len(test_ucastnici),

# připravit deap toolbox
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# --------------- Evoluce ------------------ #
pop = toolbox.population(n=200)
algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, verbose=False)

best = tools.selBest(pop, 1)[0]
best_score = toolbox.evaluate(best)[0]

# vypsání výsledků
print("------------------------------")
print("Evoluční generování dokončeno")
print("------------------------------")

print("Data")
print(f"Genom: {best}")
print(f"Score: {best_score}")
print("------------------------------")

print("Zapisuji genom ...")
write_best_genom(best, test_ucastnici, best_score)
print("Dokončeno ...")


------------------------------
Evoluční generování dokončeno
------------------------------
Data
Genom: [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
Score: 257.25
------------------------------
Zapisuji genom ...
Dokončeno ...


## Moje funkce

funkce pracuje na práci s genomvou tabulkou, ale snaží se splňovat
1. Být hodný (nezačíná zradou)
2. Odpouští (pokud oponent přestane zrazovat, odpustí)
3. Pokud zrazen hned oplatí (nenechává se zneužívat)
4. Strategie je jasná (omezení náhodnosti)

In [15]:


# -------------- Funkce Reaktivního Agenta ------------- #
best = load_best_genom()["genom"]

if not best:
    print("Agent is not trained yet. Please train it first.")
    exit(1)

def zrada(my_history, other_history):
    round_num = len(my_history)

    # 1. Úvodní fáze: poznávání soupeře
    if round_num < 3:
        return 0  # spolupracuj pro navázání důvěry
    elif round_num < 10:
        if other_history[-1] == 1:
            return 1  # test: pokud mě zradí brzo, odpověz zradou
        return 0  # jinak dál spolupracuj

    # 2. Detekce defektora
    recent_defections = other_history[-5:].count(1)
    total_defections = other_history.count(1)
    if total_defections / round_num > 0.3 and recent_defections >= 3:
        return 1  # zraď, pokud je soupeř defektor

    # 3. Odpouštění
    if other_history[-2:] == [0, 0]:
        return 0  # odpustíme po 2 spolupracích

    # 4. Genomová tabulka (paměť 2 tahy zpět)
    idx = 8 * my_history[-2] + 4 * my_history[-1] + 2 * other_history[-2] + other_history[-1]
    return best[idx]

## Turnaj

In [16]:
import inspect
import strategies
from utils import load_best_genom, play

# -------------- Trunaj ------------- #


# seznam funkci o testování
ucastnici = [
    f for name, f in vars(strategies).items()
    if inspect.isfunction(f)
       and f.__module__ == 'strategies'
]

ucastnici.append(zrada)

STEPSNUM = 200

l = len(ucastnici)
skores = [0 for i in range(l)]

print("=========================================")
print("Turnaj")
print("hra délky:", STEPSNUM)
print("-----------------------------------------")

for i in range(l):
    for j in range(i, l):
        f1 = ucastnici[i]
        f2 = ucastnici[j]
        skore1, skore2 = play(f1, f2, STEPSNUM)
        print(f1.__name__, "x", f2.__name__, " ", skore1, ":", skore2)
        skores[i] += skore1
        skores[j] += skore2

print("=========================================")
print("= Výsledné pořadí")
print("-----------------------------------------")

# setrideni indexu vysledku
index = sorted(range(l), key=lambda k: skores[k])

poradi = 1
for i in index:
    f = ucastnici[i]
    print(poradi, ".", f.__name__, ":", skores[i])
    poradi += 1



Turnaj
hra délky: 200
-----------------------------------------
always_cooperate x always_cooperate   200 : 200
always_cooperate x random_answer   396 : 102
always_cooperate x tit_for_tat   200 : 200
always_cooperate x always_defect   600 : 0
always_cooperate x tit_for_two_tats   200 : 200
always_cooperate x grim_trigger   200 : 200
always_cooperate x biased_random   258 : 171
always_cooperate x pavlov   200 : 200
always_cooperate x probing   598 : 1
always_cooperate x soft_grudger   200 : 200
always_cooperate x alternator   400 : 100
always_cooperate x generous_tit_for_tat   200 : 200
always_cooperate x zrada   200 : 200
random_answer x random_answer   270 : 315
random_answer x tit_for_tat   285 : 288
random_answer x always_defect   504 : 192
random_answer x tit_for_two_tats   216 : 351
random_answer x grim_trigger   497 : 203
random_answer x biased_random   210 : 327
random_answer x pavlov   299 : 299
random_answer x probing   510 : 177
random_answer x soft_grudger   394 : 253
random